In [ ]:
%pip install numpy scikit-learn

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [ ]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [ ]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [ ]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [ ]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [ ]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [ ]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [ ]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [ ]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [ ]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


## Primer punto

Para el primer punto creo una función para generar las muestras y buscar los documentos mas similares.

In [ ]:

def encontrar_mas_cercanos(X_train, y_train, newsgroups_train):

  # Seleccionar un documento al azar
  idx = np.random.randint(0, X_train.shape[0])
  print(f"-------------- Indice del documento original: {idx} --------------")
  # Calcular la similaridad coseno con todos los documentos
  cossim = cosine_similarity(X_train[idx], X_train)[0]

  # Encontrar los 5 documentos más similares
  mostsim = np.argsort(cossim)[::-1][1:6]

  # Imprimir la categoría del documento original
  print(f"Documento original: categoría {newsgroups_train.target_names[y_train[idx]]}")
  print(newsgroups_train.data[idx])

  # Imprimir las categorías y el contenido de los 5 documentos más similares
  for i in mostsim:

    print(f"-------------- Indice del documento relacionado: {i} --------------" )
    print(f"\nDocumento similar: categoría {newsgroups_train.target_names[y_train[i]]}")
    print(newsgroups_train.data[i])




In [ ]:
encontrar_mas_cercanos(X_train, y_train, newsgroups_train)

-------------- Indice del documento original: 5297 --------------
Documento original: categoría comp.windows.x
Could some kind soul point me in the right direction for the
FAQ list for this group.

Thanks.

--------------------
Saad Mufti
Personal Library Software
-------------- Indice del 3355 documento mas cercano: 3355 --------------

Documento similar: categoría rec.motorcycles

Like, there's a FAQ for this?

-------------- Indice del 9315 documento mas cercano: 9315 --------------

Documento similar: categoría comp.windows.x
Is there any FAQ list for Programming in X windows? 
-------------- Indice del 8576 documento mas cercano: 8576 --------------

Documento similar: categoría rec.sport.baseball
Could some kind soul out there e-mail me the 411 on where I can find the mlb.c  
program?  I'm interested in some road trips this year....

-------------- Indice del 3102 documento mas cercano: 3102 --------------

Documento similar: categoría rec.motorcycles
Could some kind soul tell me

**Documento 5297**

 Solicita información sobre FAQ, los documentos relacionados 3 hablan explicitamente sobre FAQs y de los otros dos se pide información usando el mismo modismo "Could some kind soul".

In [ ]:
encontrar_mas_cercanos(X_train, y_train, newsgroups_train)

-------------- Indice del documento original: 6968 --------------
Documento original: categoría talk.politics.misc
# #Slavery makes economic sense (it NEVER makes MORAL sense) when human
# #muscle power is an economically valuable asset. Agricultural slavery might
# #have lasted right up to the first mechanical cotton reaper, but no
# #further; reapers are cheaper than slaves, and don't have to be fed during
# #the winter. 
# 
# This argument makes a several fundamental errors.  First "agricultural"
# slavery was not limited to production of cotton.  In the American south
# slave labor was used extensively to grow tobacco, sugar, and rice, all
# of which remained labor intensive enterprises well into the 20th century.

And of course, in states like Kentucky and Virginia, not well-suited to
large-scale plantations, slave labor was used to make one of the most 
valuable agricultural products of all: more slaves.  In some ways, this
treatment of humans beings as breeding livestock is the 

**Documento 5297**

El documento original tiene una postura bastante conservadora sobre la esclavitud. Los dos documentos mas cercanos, 9966 y 6234, son relacionado a la esclavitud como era de esperarse, pero de ahí en adelante se ve una relación subyacente.

Los dos documentos que siguen, 913 y 2392, son declaraciones antisemitas por lo tanto se puede ver que aunque no este hablando del tema se conserva de alguna forma la idelogía de los autores.

El documento que más me intereso fue el último, 6880, porque es una defensa de la ocupación británica de las islas malvinas. Me sorprendio la similitud que encontró para sustentar que el colonialismo es extensión del esclavismo.

In [ ]:

encontrar_mas_cercanos(X_train, y_train, newsgroups_train)

-------------- Indice del documento original: 5551 --------------
Documento original: categoría comp.os.ms-windows.misc
If a PC has one, does Windows 3.1 use a math co-processor?  I'm not talking
about specific apps, but the OS (if you want to call it that) itself?

Please respond by email.
-------------- Indice del documento relacionado: 820 --------------

Documento similar: categoría comp.os.ms-windows.misc
This is the official Request for Discussion (RFD) for the creation of two
new newsgroups for Microsoft Windows NT.  This is a second RFD, replacing
the one originally posted in January '93 (and never taken to a vote).  The
proposed groups are described below:

NAME: 	 comp.os.ms-windows.nt.setup
STATUS:  Unmoderated.
PURPOSE: Discussions about setting up and installing Windows NT, and about
	 system and peripheral compatability issues for Windows NT.

NAME:	 comp.os.ms-windows.nt.misc
STATUS:	 Unmoderated.
PURPOSE: Miscellaneous non-programming discussions about using Windows NT,

**Documento 5551**
Tanto el documento original como  todos los relacionado hablando sobre sistemas operativos, principalmente Windows. Tiene sentido que todos caigan dentro de la categoría "comp.os.ms-windows.misc"

In [ ]:
encontrar_mas_cercanos(X_train, y_train, newsgroups_train)

-------------- Indice del documento original: 10219 --------------
Documento original: categoría sci.space

: >There is an emergency oxygen system that is capable of maintaining a
: >breathable atmosphere in the cabin for long enough to come down, even
: >if there is something like a 5cm hole in the wall that nobody tries
: >to plug.

Josh Hopkins (jbh55289@uxa.cso.uiuc.edu) replied:
: Wow.

: Double wow.  Can you land a shuttle with a 5cm hole in the wall?

Personnally, I don't know, but I'd like to try it sometime.

Programmatically, yes, we can land an Orbiter with a 5 cm hole in
the wall -- provided that the thing which caused 5 cm hole didn't
cause a Crit 1 failure on some of the internal systems.  There are
a few places where a 5 cm hole would cause a Bad Day -- especially
if the 5 cm hole went all the way through the Orbiter and out the
other side, as could easily happen with a meteor strike.  But a
hole in the pressure vessel would cause us to immediately de-orbit
to the next a

**Documento 10219**

El documento principal habla sobre la posibilidad de alunizar con un agüjero en la pared de una nave espacial. El documento más cercano, 7425, habla sobre agujeros pero en un auto, supongo que es una estimación correcta porque pero menos significativa que el resto de los documentos cercanos que hablan sobre viajes en el espacio.  


In [ ]:
encontrar_mas_cercanos(X_train, y_train, newsgroups_train)

-------------- Indice del documento original: 7648 --------------
Documento original: categoría comp.sys.mac.hardware







Also, has anyone heard any rumors that the new docks (the ones with the CPU
:-) will be better designed that this first batch?  I love my Duo, but
installing cards in the dock is not much fun.

-Bob
-------------- Indice del documento relacionado: 1684 --------------

Documento similar: categoría comp.sys.mac.hardware
Has anyone had any problems with their Duo Dock not ejecting the Duo
properly?

When I first got it, the Duo would come out of the Dock a couple of inches
when ejected, and I had to pull it the rest of the way. Nowadays (and I've
had the system for 4 months), the Duo doesn't come out *at* *all* - despite
the fact that the mechanism makes all the appropriate noises, and I have to
grab hold of it and pull it out myself. Is there a simple fix for this, or
do I have to return it to my Apple Dealer, where it will languish for weeks
while I have to make d

**Documento  7648**

Tanto el documento original como todos los cercanos hablan sobre el PowerBook Duo, un modelo de Apple. Los dos documentos mas cercanos comparte tambien la especificidad sobre el problema en si con ese modelo el "Duo Dock", supongo que es un tipo de entrada que inventaron para el modelo y salió mal.

